In [26]:
import datetime
import sys
import os

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import itertools
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from cond_rnn import ConditionalRNN
from sklearn.preprocessing import MinMaxScaler
from pmdarima.arima import auto_arima

import src.preprocessing
from src.functions import load_data, mape, TimeSeriesTensor, create_evaluation_df, series_to_supervised, plot_train_history, validation

np.set_printoptions(threshold=sys.maxsize)
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [27]:
csv_path = '../data/Dieteren_case/Consumption_15min.csv'
df = pd.read_csv(csv_path, index_col=0)

In [28]:
df.index = pd.to_datetime(df.index)
df = df.resample('H').mean()

In [29]:
def train_test_split(df, n_test):
    test_df = df.copy()[-n_test:]
    train_df = df.copy()[:len(test_df)]
    return train_df, test_df

In [30]:
def ConditionalRNN_data_preparation(df, n_test=4380, T=24, HORIZON=24):
    df = src.preprocessing.preprocess(df, 'Belgium')
    c2 = series_to_supervised(df)
    c1 = df.iloc[:,1:9]
    c1 = c1.loc[c2.index]
    short_df = df.iloc[:,[0,1,-4,-3,-2,-1]].copy()
    #check how to formulate more correctly
    short_df = short_df[146:]
    train_df, test_df = train_test_split(short_df, n_test)
    train_c1, test_c1 = train_test_split(c1, n_test)
    train_c2, test_c2 = train_test_split(c2, n_test)
    y_scaler = MinMaxScaler()
    y_scaler.fit(train_df[['value']])
    train_c2[['value(t-24)']] = y_scaler.fit_transform(train_c2[['value(t-24)']])
    train_c2[['value(t-168)']] = y_scaler.fit_transform(train_c2[['value(t-168)']])
    X_scaler = MinMaxScaler()
    train_df[train_df.columns] = X_scaler.fit_transform(train_df)
    test_df[train_df.columns] = X_scaler.fit_transform(test_df)
    tensor_structure = {'X':(range(-T+1, 1), ['value','fractional hour_sin','fractional hour_cos','day of year_sin','day of year_cos'])}
    train_inputs = TimeSeriesTensor(train_df, 'value', HORIZON, tensor_structure)
    test_inputs = TimeSeriesTensor(test_df, 'value', HORIZON, tensor_structure)
    train_c1 = c1.reindex(train_inputs.dataframe.index)
    train_c2 = c2.reindex(train_inputs.dataframe.index)
    test_c1 = c1.reindex(test_inputs.dataframe.index)
    test_c2 = c2.reindex(test_inputs.dataframe.index)
    return train_inputs, test_inputs, train_c1, test_c1, train_c2, test_c2, y_scaler

In [35]:
def MIMO_fulldata_preparation(df, n_test=4380, T=24, HORIZON=24):
    df = src.preprocessing.preprocess(df, 'Belgium')
    df = df.merge(series_to_supervised(df), how='right', left_index=True, right_index=True)
    train_df, test_df = train_test_split(df, n_test)
    y_scaler = MinMaxScaler()
    y_scaler.fit(train_df[['value']])
    long_scaler = MinMaxScaler()
    test_df[test_df.columns] = long_scaler.fit_transform(test_df)
    train_df[train_df.columns] = long_scaler.fit_transform(train_df)
    tensor_structure = {'X':(range(-T+1, 1), train_df.columns)}
    train_inputs = TimeSeriesTensor(train_df, 'value', HORIZON, tensor_structure)
    test_inputs = TimeSeriesTensor(test_df, 'value', HORIZON, tensor_structure)
    return train_inputs, test_inputs, y_scaler

In [31]:
NUM_CELLS = 32
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 24
# ConditionalRNN with the functional API.
#i = Input(shape=[HORIZON, 5], name='time-series')
#c1 = Input(shape=[8], name='one-hot')
#c2 = Input(shape=[2], name='lags-as-features')
# add the condition tensor here.
#x = ConditionalRNN(NUM_CELLS, cell='LSTM', name='cond_rnn_0')([i, c1, c2])
# Dense layer to output
#x = Dense(HORIZON)(x)
#ConditionalRNN = Model(inputs=[i, c1, c2], outputs=[x])

ConditionalRNN = Sequential(layers=[ConditionalRNN(NUM_CELLS, cell='LSTM'),
                                    Dense(HORIZON)])


train_inputs, test_inputs, train_c1, test_c1, train_c2, test_c2, y_scaler = ConditionalRNN_data_preparation(df, n_test=1465)
# full data LSTM MIMO compilation and fit
ConditionalRNN.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')

history = ConditionalRNN.fit([train_inputs['X'],train_c1,train_c2], train_inputs['target'], batch_size=32, epochs=MAX_EPOCHS,
              validation_split=0.15,
              callbacks=[early_stopping], verbose=1)

#save_model(ConditionalRNN, i)
predictions = ConditionalRNN.predict([test_inputs['X'],test_c1,test_c2])
eval_df = create_evaluation_df(predictions, test_inputs, HORIZON, y_scaler)
mae = validation(eval_df['prediction'], eval_df['actual'], 'MAE')
rmse = validation(eval_df['prediction'], eval_df['actual'], 'RMSE')

Epoch 1/100
Consider rewriting this model with the Functional API.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by pass

38/38 [==============================] - 1s 15ms/step - loss: 0.0243 - mean_squared_error: 0.0243 - val_loss: 0.0530 - val_mean_squared_error: 0.0530
Epoch 36/100
38/38 [==============================] - 1s 14ms/step - loss: 0.0234 - mean_squared_error: 0.0234 - val_loss: 0.0546 - val_mean_squared_error: 0.0546
Epoch 37/100
38/38 [==============================] - 1s 14ms/step - loss: 0.0225 - mean_squared_error: 0.0225 - val_loss: 0.0539 - val_mean_squared_error: 0.0539
Epoch 38/100
38/38 [==============================] - 1s 15ms/step - loss: 0.0224 - mean_squared_error: 0.0224 - val_loss: 0.0556 - val_mean_squared_error: 0.0556
Epoch 39/100
38/38 [==============================] - 1s 15ms/step - loss: 0.0219 - mean_squared_error: 0.0219 - val_loss: 0.0553 - val_mean_squared_error: 0.0553
Epoch 40/100
38/38 [==============================] - 1s 15ms/step - loss: 0.0214 - mean_squared_error: 0.0214 - val_loss: 0.0535 - val_mean_squared_error: 0.0535
Epoch 41/100
38/38 [===============

In [32]:
print('rmse {}'.format(rmse))
print('mae {}'.format(mae))

rmse 144.85390394635806
mae 99.88938762305449


In [42]:
eval_df.to_csv('Dieteren-forecast_2months.csv')

In [39]:
NUM_CELLS = 32
MAX_EPOCHS = 100
BATCHSIZE = 32
patience = 10
HORIZON = 24

LSTMIMO = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, input_shape=(24, 15)),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(HORIZON)
])

train_inputs, test_inputs, y_scaler = MIMO_fulldata_preparation(df)
# full data LSTM MIMO compilation and fit
LSTMIMO.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')

history = LSTMIMO.fit(train_inputs['X'], train_inputs['target'], batch_size=32, epochs=MAX_EPOCHS, validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)

predictions = LSTMIMO.predict(test_inputs['X'])
eval_df = create_evaluation_df(predictions, test_inputs, HORIZON, y_scaler)
mae = validation(eval_df['prediction'], eval_df['actual'], 'MAE')
rmse = validation(eval_df['prediction'], eval_df['actual'], 'RMSE')

Epoch 1/100
116/116 [==============================] - 2s 17ms/step - loss: 0.0908 - mean_squared_error: 0.0908 - val_loss: 0.0495 - val_mean_squared_error: 0.0495
Epoch 2/100
116/116 [==============================] - 1s 13ms/step - loss: 0.0444 - mean_squared_error: 0.0444 - val_loss: 0.0440 - val_mean_squared_error: 0.0440
Epoch 3/100
116/116 [==============================] - 1s 13ms/step - loss: 0.0371 - mean_squared_error: 0.0371 - val_loss: 0.0371 - val_mean_squared_error: 0.0371
Epoch 4/100
116/116 [==============================] - 1s 12ms/step - loss: 0.0281 - mean_squared_error: 0.0281 - val_loss: 0.0329 - val_mean_squared_error: 0.0329
Epoch 5/100
116/116 [==============================] - 1s 13ms/step - loss: 0.0253 - mean_squared_error: 0.0253 - val_loss: 0.0328 - val_mean_squared_error: 0.0328
Epoch 6/100
116/116 [==============================] - 1s 12ms/step - loss: 0.0241 - mean_squared_error: 0.0241 - val_loss: 0.0297 - val_mean_squared_error: 0.0297
Epoch 7/100
116/

In [40]:
print('rmse {}'.format(rmse))
print('mae {}'.format(mae))

rmse 144.90271518363247
mae 99.66371330971452
